In [ ]:
#Upgrade quiet quiet
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *
import os

# Gathering Data

In [ ]:
path = Path('seg_train/seg_train')

In [ ]:
path

In [ ]:
#fns file names
fns = get_image_files(path)

In [ ]:
#doc(get_image_files)

In [ ]:
fns

In [ ]:
# verify images
failed = verify_images(fns)

In [ ]:
failed
#if some images failed to verify, use failed.map(Ptah.unlink);

# From Data To Dataloaders

In [ ]:
#fastai needs four things
#kind of data, how to get the items, validation and label
#x the independent variable ImageBlock
#y the dependent variable CategoryBlock
#images should be equal in size for a tensor
scenes = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
#doc(DataBlock)

In [ ]:
#dls dataloaders
dls = scenes.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=6, nrows=2)

# Data Agumentation

In [ ]:
scenes = scenes.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = scenes.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(6, nrows=2)

In [ ]:
uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
img

In [ ]:
learn.predict(img)

In [ ]:
learn.dls.vocab

In [ ]:
learn.export(fname="intel.pkl")